In [1]:
!pip install pymongo pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 13.8 MB/s eta 0:00:00


In [2]:
!pip install ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.7 MB/s eta 0:00:00


In [20]:
import hashlib
import pickle
import pandas as pd
from flask import Flask, render_template, request, redirect, url_for, session, flash
from pymongo import MongoClient
from sklearn.preprocessing import StandardScaler
from pyngrok import ngrok  # Import ngrok
from urllib.parse import quote_plus # Import quote_plus
import joblib


ngrok.set_auth_token("2os4BpqJxJ2zMlPIKnogyPJesjm_7PXoEfByyvcRCQ6RUaz87")
app = Flask(__name__)
app.secret_key = 'MANJU'  # Replace with a secure key

# MongoDB Atlas connection
username = "manjunath"  # Replace with your MongoDB Atlas username
password = "manju@2005"  # Replace with your MongoDB Atlas password
cluster_url = "cluster0.9gafb.mongodb.net"  # Replace with your MongoDB Atlas cluster URL
db_name = "Project0"  # Database name

# URL-encode the username and password to comply with RFC 3986
encoded_username = quote_plus(username)
encoded_password = quote_plus(password)

# Create connection string and client for MongoDB Atlas with escaped username and password
uri = f"mongodb+srv://{encoded_username}:{encoded_password}@{cluster_url}/{db_name}?retryWrites=true&w=majority"
client = MongoClient(uri)  # Initialize MongoClient
db = client.get_database(db_name)  # Replace with your database name
users_collection = db.users  # Your collection name for users

# Load your pre-trained model and scaler (update paths if necessary)
model = pickle.load(open('random_forest_model.pkl', 'rb'))
feature_names = ['N', 'P', 'K', 'pH', 'EC','OC', 'S', 'Zn', 'Fe', 'Cu', 'Mn', 'B']
# ... (rest of your code) ...
# Route to redirect to the login page (This is the first page that will be opened)
@app.route('/')
def index():
    return redirect(url_for('login'))  # Redirect to the login page by default

# Route to handle login page
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        # Handle form submission
        username = request.form['username']
        password = request.form['password']

        # Hash the password for comparison
        hashed_password = hashlib.sha256(password.encode()).hexdigest()

        # Check if the user exists in MongoDB
        user = users_collection.find_one({"username": username, "password": hashed_password})

        if user:
            # Set the session data for logged-in user
            session['username'] = username
            flash("Login successful!")
            return redirect(url_for('ProjectHomepage'))  # Redirect to home page on successful login
        else:
            flash("Invalid login credentials. Please try again.")
            return redirect(url_for('login'))  # Redirect back to login page on failure

    return render_template('login.html')  # Display login form when GET request

# Route to handle user sign-up
@app.route('/signup', methods=['POST'])
def signup():
    username = request.form['username']
    password = request.form['password']
    confirm_password = request.form['confirm_password']

    # Check if passwords match
    if password != confirm_password:
        flash("Passwords do not match!")
        return redirect(url_for('login'))

    # Hash the password for security
    hashed_password = hashlib.sha256(password.encode()).hexdigest()

    # Check if username already exists
    existing_user = users_collection.find_one({"username": username})
    if existing_user:
        flash("Username already exists. Please choose another one.")
        return redirect(url_for('login'))

    # Insert the user into MongoDB
    users_collection.insert_one({"username": username, "password": hashed_password})
    flash('Registration successful! Please sign in to continue.')
    return redirect(url_for('login'))  # Redirect to login page after successful registration

# Route to handle the signin functionality for login
@app.route('/signin', methods=['POST'])
def signin():
    username = request.form['username']
    password = request.form['password']

    # Hash the password
    hashed_password = hashlib.sha256(password.encode()).hexdigest()

    # Check user credentials in MongoDB
    user = users_collection.find_one({"username": username, "password": hashed_password})

    if user:
        # Store user session data
        session['username'] = username
        flash("Signin successful!")
        return redirect(url_for('ProjectHomepage'))  # Redirect to homepage
    else:
        flash("Invalid credentials! Please try again.")
        return redirect(url_for('login'))  # Redirect back to login page if authentication fails

# Route to handle the project homepage after login
@app.route('/ProjectHomepage', methods=['GET', 'POST'])
def ProjectHomepage():
    # Ensure that the user is logged in before accessing this page
    if 'username' not in session:
        flash("Please login first.")
        return redirect(url_for('login'))

    prediction = None

    # Check if the form for prediction is submitted
    if request.method == 'POST':
        try:
            # Collect input data from form and ensure conversion to floats
            input_values = [
                float(request.form['input_1']),
                float(request.form['input_2']),
                float(request.form['input_3']),
                float(request.form['input_4']),
                float(request.form['input_5']),
                float(request.form['input_6']),
                float(request.form['input_7']),
                float(request.form['input_8']),
                float(request.form['input_9']),
                float(request.form['input_10']),
                float(request.form['input_11']),
                float(request.form['input_12']),
            ]

            # Debugging output
            print(f"Received Input Values: {input_values}")

            # Create a DataFrame with feature names as columns for consistency with model
            input_df = pd.DataFrame([input_values], columns=feature_names)
            # Assuming you have the training data as `X_train`
            scaler = joblib.load('scaler.pkl')

            # Now you can use it for transforming new data
            input_data_scaled = scaler.transform(input_df) if scaler else input_df
            # Scale the data if a scaler is available
            input_data_scaled = scaler.transform(input_df) if scaler else input_df

            # Make prediction using the model
            prediction = model.predict(input_data_scaled)[0]

        except Exception as e:
            flash(f"Error during prediction: {e}")
            print(f"Error: {e}")

    return render_template('ProjectHomepage.html', prediction=prediction)


if __name__ == "__main__":
    port = 5000
    public_url = ngrok.connect(port)
    print(f'Ngrok URL: {public_url}')
    app.run(port=port, debug=False)  # Disable debug mode in production  # Use host='0.0.0.0' for ngrok to work properly


Ngrok URL: NgrokTunnel: "https://abd2-34-34-25-54.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [16/Nov/2024 08:48:12] "GET / HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [16/Nov/2024 08:48:14] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Nov/2024 08:48:15] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Nov/2024 08:48:34] "POST /signin HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [16/Nov/2024 08:48:35] "GET /ProjectHomepage HTTP/1.1" 200 -
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [16/Nov/2024 08:51:43] "POST /ProjectHomepage HTTP/1.1" 200 -


Received Input Values: [65.0, 12.0, 98.0, 6.9, 12.0, 2.0, 17.0, 9.0, 19.0, 12.0, 34.0, 0.8]


In [8]:
import requests
import json

try:
    response = requests.post('https://d41a-34-34-25-54.ngrok-free.app/predict', headers={'Content-Type': 'application/json'}, data=json.dumps({'input': 'test'}))
    response.raise_for_status() # Raise an exception for non-200 status codes
    data = response.json()
    print(data)
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON response: {e}")

Error: 404 Client Error: Not Found for url: https://d41a-34-34-25-54.ngrok-free.app/predict
